In [11]:
%pip install tables


   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   - -------------------------------------- 0.2/4.4 MB 4.1 MB/s eta 0:00:02
   ------ --------------------------------- 0.7/4.4 MB 7.0 MB/s eta 0:00:01
   ----------- ---------------------------- 1.2/4.4 MB 8.6 MB/s eta 0:00:01
   ---------------- ----------------------- 1.8/4.4 MB 9.4 MB/s eta 0:00:01
   --------------------- ------------------ 2.3/4.4 MB 9.8 MB/s eta 0:00:01
   -------------------------- ------------- 2.9/4.4 MB 10.2 MB/s eta 0:00:01
   ------------------------------- -------- 3.5/4.4 MB 10.5 MB/s eta 0:00:01
   ------------------------------------ --- 4.0/4.4 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  4.4/4.4 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------  4.4/4.4 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------  4.4/4.4 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 4.4/4.4 MB 8.0 MB/s eta 0:00:00
   ---------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('temps.xlsx')
df.head()

,year,month,day,week,temp_2,temp_1,average,actual
0,2016,1,1,Fri,45,45,45.6,45
1,2016,1,2,Sat,44,45,45.7,44
2,2016,1,3,Sun,45,44,45.8,41
3,2016,1,4,Mon,44,41,45.9,40
4,2016,1,5,Tues,41,40,46.0,44


In [4]:
fet = pd.get_dummies(df)
fet.head()

,year,month,day,temp_2,temp_1,average,actual,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,True,False,False,False,False,False,False
1,2016,1,2,44,45,45.7,44,False,False,True,False,False,False,False
2,2016,1,3,45,44,45.8,41,False,False,False,True,False,False,False
3,2016,1,4,44,41,45.9,40,False,True,False,False,False,False,False
4,2016,1,5,41,40,46.0,44,False,False,False,False,False,True,False


In [5]:
labels = np.array(fet['actual'])
fet = fet.drop(columns='actual')

fet_list = list(fet.columns)
fet = np.array(fet)

In [6]:
from sklearn.model_selection import train_test_split

train_fet, test_fet, train_labels, test_labels = train_test_split(fet, labels, train_size=0.25, random_state=42)

In [7]:
import sklearn.metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor


In [8]:
gbr = GradientBoostingRegressor()
gbr.fit(train_fet, train_labels)
gbr_pred = gbr.predict(test_fet)

In [10]:
r_sq = gbr.score(test_fet, test_labels)
r_sq

0.7231295915632026

In [12]:
from tables.file import parameters
from sklearn.model_selection import GridSearchCV

parameters = {'learning_rate' : [0.03],
              'subsample' : [0.2],
              'n_estimators' : [100, 500, 1000, 1500],
              'max_depth' : [8]


}

grid_search = GridSearchCV(gbr, parameters, scoring= 'r2', cv=2, n_jobs= -1 )


In [13]:
grid_search.fit(train_fet, train_labels)

GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.03], 'max_depth': [8],
                         'n_estimators': [100, 500, 1000, 1500],
                         'subsample': [0.2]},
             scoring='r2')

In [15]:
grid_search.best_estimator_

GradientBoostingRegressor(learning_rate=0.03, max_depth=8, subsample=0.2)

In [17]:
best_model = grid_search.best_estimator_
best_model.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.03,
 'loss': 'squared_error',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 0.2,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [20]:
gbr_tunned = GradientBoostingRegressor(alpha = 0.9,
 ccp_alpha = 0.0,
 criterion = 'friedman_mse',
 init= None,
 learning_rate= 0.03,
 loss= 'squared_error',
 max_depth= 8,
 max_features= None,
 max_leaf_nodes= None,
 min_impurity_decrease= 0.0,
 min_samples_leaf= 1,
 min_samples_split= 2,
 min_weight_fraction_leaf= 0.0,
 n_estimators= 100,
 n_iter_no_change= None,
 random_state= None,
 subsample= 0.2,
 tol= 0.0001,
 validation_fraction= 0.1,
 verbose= 0,
 warm_start= False)

In [25]:
gbr_tunned.fit(train_fet, train_labels)
gbr_tunned_pred = gbr_tunned.predict(test_fet)

In [26]:
r_sq = gbr_tunned.score(test_fet, test_labels)
r_sq

0.796430070041116